In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv
/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv


In [2]:
train1 = pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
test1 = pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission = pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [3]:
train1.head(5)

,Country Code,Country Name,Indicator,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,Afg,Afghanistan,Access to electricity (% of population),4.446890831,9.294527054,14.13361645,18.9711647,23.81418228,28.66967201,33.54441833,38.44000244,42.4,48.27900696,42.7,43.22201891,69.1,68.0408783,89.5,71.5
1,Afg,Afghanistan,Agricultural land (% of land area),57.94581666,57.94734986,57.93968385,58.08380479,58.15126566,58.13440044,58.12366803,58.12980084,58.13286724,58.13286724,58.13440044,58.13133404,58.12980084,58.12366803,58.12366803,58.12366803
2,Afg,Afghanistan,"Annual freshwater withdrawals, total (% of int...",43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668,43.01590668
3,Afg,Afghanistan,Arable land (% of land area),11.77958696,11.77958696,11.77192095,11.91604189,11.98350275,11.96663754,11.94977232,11.94977232,11.94977232,11.94823912,11.94823912,11.94517272,11.94363951,11.93597351,11.91450869,11.90530948
4,Afg,Afghanistan,Forest area (% of land area),1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994,1.852781994


In [4]:
test1.head()

,Country Name,Indicator,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,Access to electricity (% of population),97.7,97.7,93.43087769,97.7,97.7
1,Afghanistan,Agricultural land (% of land area),58.12366803,58.12366803,58.27698818,58.27698818,58.74154823
2,Afghanistan,"Annual freshwater withdrawals, total (% of int...",43.01590668,43.01590668,43.01590668,43.01590668,43.01590668
3,Afghanistan,Arable land (% of land area),11.85011422,11.80411818,11.94977232,11.94057311,12.00343437
4,Afghanistan,Forest area (% of land area),1.852781994,1.852781994,1.852781994,1.852781994,1.852781994


In [5]:
train1.Indicator.unique()

array(['Access to electricity (% of population)',
       'Agricultural land (% of land area)',
       'Annual freshwater withdrawals, total (% of internal resources)',
       'Arable land (% of land area)', 'Forest area (% of land area)',
       'Electric power consumption (kWh per capita)',
       'Energy use (kg of oil equivalent per capita)',
       'Renewable electricity output (% of total electricity output)',
       'Renewable energy consumption (% of total final energy consumption)',
       'Population growth (annual %)', 'GDP per capita (current US$)',
       'CO2 emissions (metric tons per capita)'], dtype=object)

In [6]:
test1.Indicator.unique()

array(['Access to electricity (% of population)',
       'Agricultural land (% of land area)',
       'Annual freshwater withdrawals, total (% of internal resources)',
       'Arable land (% of land area)', 'Forest area (% of land area)',
       'Electric power consumption (kWh per capita)',
       'Energy use (kg of oil equivalent per capita)',
       'Renewable electricity output (% of total electricity output)',
       'Renewable energy consumption (% of total final energy consumption)',
       'Population growth (annual %)', 'GDP per capita (current US$)'],
      dtype=object)

In [7]:
#[year[0: 4] for year in train1.columns[3: ]]


**Modify Dataframe**

In [8]:
def transFormDF(df, df_type):
    if df_type == 'train':
        df = df.drop(['Country Code'], axis = 1)
        years = [year[0: 4] for year in df.columns[2: ]]
    else:
        years = [year[0: 4] for year in df.columns[2: ]]
        
    df = df.pivot(index = 'Country Name', columns = 'Indicator')
    
    data_frames = []
    for year in years:
        mapp = {}
        for col in df.columns:
            if year in col[0]:
                mapp[col[1]] = df[col]
        data = pd.DataFrame(mapp)
        data['year'] = year
        data_frames.append(data)
    
    new_df = pd.concat(data_frames, axis = 0)
    return new_df
    
    

In [9]:
train2 = transFormDF(train1, 'train')
test2 = transFormDF(test1, 'test')

In [10]:
#test2['Energy use (kg of oil equivalent per capita)']

In [11]:
#train2.index #'Country Name' Column
train2 = train2.reset_index() # index_column --> attribute
test2 = test2.reset_index()

In [12]:
train3 = train2[['Country Name', 'year', 'CO2 emissions (metric tons per capita)']]
test3 = test2[['Country Name', 'year']]

In [13]:
train3.head()

,Country Name,year,CO2 emissions (metric tons per capita)
0,Afghanistan,2000,0.055166607
1,Africa Eastern and Southern,2000,0.891352058
2,Africa Western and Central,2000,0.521688598
3,Albania,2000,1.031567545
4,Algeria,2000,2.601065339


In [14]:
test3.head()

,Country Name,year
0,Afghanistan,2016
1,Africa Eastern and Southern,2016
2,Africa Western and Central,2016
3,Albania,2016
4,Algeria,2016


In [15]:
train3 = train3.sort_values(by = ['Country Name', 'year'])
test3 = test3.sort_values(by = ['Country Name', 'year'])

In [16]:
#train3.head()
#test3.head()

**Missing Value Checking**

In [17]:
train3.isnull().sum()

Country Name                              0
year                                      0
CO2 emissions (metric tons per capita)    0
dtype: int64

In [18]:
test3.isnull().sum()

Country Name    0
year            0
dtype: int64

In [19]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4256 entries, 0 to 4255
Data columns (total 3 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Country Name                            4256 non-null   object
 1   year                                    4256 non-null   object
 2   CO2 emissions (metric tons per capita)  4256 non-null   object
dtypes: object(3)
memory usage: 133.0+ KB


****Convert datatype object to numeric****

In [20]:
train3[train3.columns[1: ]] = train3[train3.columns[1: ]].apply(pd.to_numeric, errors='coerce')
test3['year'] = pd.to_numeric(test3['year'], errors='coerce')

In [21]:
#train3.head()
#test3.head()

In [22]:
#train3.info()
#test3.info()

In [23]:
train3.isnull().sum()
#test3.isnull().sum()

Country Name                                0
year                                        0
CO2 emissions (metric tons per capita)    432
dtype: int64

**Fill Up Null values Using Mean**

In [24]:
train3['CO2 emissions (metric tons per capita)'].fillna(train3['CO2 emissions (metric tons per capita)'].mean(), inplace = True)
    


/tmp/ipykernel_18/1636053970.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train3['CO2 emissions (metric tons per capita)'].fillna(train3['CO2 emissions (metric tons per capita)'].mean(), inplace = True)


In [25]:
train3.isnull().sum()

Country Name                              0
year                                      0
CO2 emissions (metric tons per capita)    0
dtype: int64

In [26]:
#train3.info()

**Encoding Country Name Column**

In [27]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

train3['Country Name'] = label_encoder.fit_transform(train3['Country Name']) 
test3['Country Name'] = label_encoder.fit_transform(test3['Country Name']) 

In [28]:
#train3.head()
#test3.head()

**Model Building**

In [29]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [30]:
Y_train = train3['CO2 emissions (metric tons per capita)']
X_train = train3.drop(['CO2 emissions (metric tons per capita)'], axis = 1)

In [31]:
model.fit(X_train, Y_train)

LinearRegression()

In [32]:
model.score(X_train, Y_train)

0.000473661573353823

In [33]:
Y_pred = model.predict(test3)

In [34]:
Y_pred

array([4.1061207 , 4.11019639, 4.11427209, ..., 4.48205364, 4.48612933,
       4.49020503])

In [35]:
sample_submission.head()

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,3.2253,1.1712,3.7364,0.6141,1.9883,0.0942
1,Albania,2.3565,2.3681,2.1690,1.5303,3.5665,1.7349
2,Algeria,1.0482,1.2839,1.5509,2.4556,3.0417,0.0114
3,American Samoa,3.5276,3.1392,1.1129,1.6479,3.1007,0.6840
4,Andorra,0.8961,3.9242,0.0745,2.0379,3.0965,2.1794


**Making New Dataframe To Predict**

In [36]:
countries = test1['Country Name'].unique().tolist()
years = [2016, 2017, 2018, 2019, 2020, 2030]
country_year = []
for country in countries:
    for year in years:
        s = country + '_' + str(year)
        country_year.append(s)

In [37]:
test_df = pd.DataFrame({'Country Name': country_year})

In [38]:
test_df.head()

,Country Name
0,Afghanistan_2016
1,Afghanistan_2017
2,Afghanistan_2018
3,Afghanistan_2019
4,Afghanistan_2020


In [39]:
test_df['year'] = test_df['Country Name'].str[-4: ]

In [40]:
test_df['Country Name'] = test_df['Country Name'].str[: -5]

In [41]:
test_df.head()

,Country Name,year
0,Afghanistan,2016
1,Afghanistan,2017
2,Afghanistan,2018
3,Afghanistan,2019
4,Afghanistan,2020


In [42]:
countries = test_df['Country Name']

In [43]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  1596 non-null   object
 1   year          1596 non-null   object
dtypes: object(2)
memory usage: 25.1+ KB


In [44]:
test_df['year'] = pd.to_numeric(test_df['year'])

In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  1596 non-null   object
 1   year          1596 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.1+ KB


In [46]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

test_df['Country Name'] = label_encoder.fit_transform(test_df['Country Name'])

In [47]:
test_df.head()

,Country Name,year
0,0,2016
1,0,2017
2,0,2018
3,0,2019
4,0,2020


In [48]:
predicted_values = model.predict(test_df)

In [49]:
predicted_values

array([4.1061207 , 4.11019639, 4.11427209, ..., 4.48196577, 4.48604146,
       4.52679841])

In [50]:
test_df['CO2 Emissions'] = predicted_values

In [51]:
test_df.head()

,Country Name,year,CO2 Emissions
0,0,2016,4.106121
1,0,2017,4.110196
2,0,2018,4.114272
3,0,2019,4.118348
4,0,2020,4.122423


In [52]:
test_df.head()

,Country Name,year,CO2 Emissions
0,0,2016,4.106121
1,0,2017,4.110196
2,0,2018,4.114272
3,0,2019,4.118348
4,0,2020,4.122423


In [53]:
test_df['Country Name'] = countries

In [54]:
test_df

,Country Name,year,CO2 Emissions
0,Afghanistan,2016,4.106121
1,Afghanistan,2017,4.110196
2,Afghanistan,2018,4.114272
3,Afghanistan,2019,4.118348
4,Afghanistan,2020,4.122423
...,...,...,...
1591,World,2017,4.473814
1592,World,2018,4.477890
1593,World,2019,4.481966
1594,World,2020,4.486041


In [55]:
final_df = test_df.pivot(index = 'Country Name', columns = 'year')

In [56]:
final_df.head()

CO2 Emissions                                \
year                                 2016      2017      2018      2019   
Country Name                                                              
Afghanistan                      4.106121  4.110196  4.114272  4.118348   
Africa Eastern and Southern      4.107509  4.111584  4.115660  4.119736   
Africa Western and Central       4.108896  4.112972  4.117048  4.121123   
Albania                          4.110284  4.114360  4.118436  4.122511   
Algeria                          4.111672  4.115748  4.119824  4.123899   

                                                 
year                             2020      2030  
Country Name                                     
Afghanistan                  4.122423  4.163180  
Africa Eastern and Southern  4.123811  4.164568  
Africa Western and Central   4.125199  4.165956  
Albania                      4.126587  4.167344  
Algeria                      4.127975  4.168732

In [57]:
final_df.columns

MultiIndex([('CO2 Emissions', 2016),
            ('CO2 Emissions', 2017),
            ('CO2 Emissions', 2018),
            ('CO2 Emissions', 2019),
            ('CO2 Emissions', 2020),
            ('CO2 Emissions', 2030)],
           names=[None, 'year'])

In [58]:
type(final_df.columns[0][1])

numpy.int64

In [59]:
#[str(col[1] + ) for col in final_df.columns]
#[  f"{col[1]} [YR{col[1]}]"     for col in final_df.columns ]

In [60]:
final_df.columns = [ f"{col[1]} [YR{col[1]}]"  for col in final_df.columns ]

In [61]:
final_df.head()

,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
Country Name,,,,,,
Afghanistan,4.106121,4.110196,4.114272,4.118348,4.122423,4.163180
Africa Eastern and Southern,4.107509,4.111584,4.115660,4.119736,4.123811,4.164568
Africa Western and Central,4.108896,4.112972,4.117048,4.121123,4.125199,4.165956
Albania,4.110284,4.114360,4.118436,4.122511,4.126587,4.167344
Algeria,4.111672,4.115748,4.119824,4.123899,4.127975,4.168732


In [62]:
final_df = final_df.reset_index()

In [63]:
final_df

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,4.106121,4.110196,4.114272,4.118348,4.122423,4.163180
1,Africa Eastern and Southern,4.107509,4.111584,4.115660,4.119736,4.123811,4.164568
2,Africa Western and Central,4.108896,4.112972,4.117048,4.121123,4.125199,4.165956
3,Albania,4.110284,4.114360,4.118436,4.122511,4.126587,4.167344
4,Algeria,4.111672,4.115748,4.119824,4.123899,4.127975,4.168732
...,...,...,...,...,...,...,...
261,West Bank and Gaza,4.468351,4.472427,4.476502,4.480578,4.484654,4.525411
262,World,4.469739,4.473814,4.477890,4.481966,4.486041,4.526798
263,"Yemen, Rep.",4.471127,4.475202,4.479278,4.483354,4.487429,4.528186
264,Zambia,4.472514,4.476590,4.480666,4.484741,4.488817,4.529574


In [64]:
#final_df.to_csv('Hacktick_53.csv', index = False)